# Necessary packages install and import

In [ ]:
pip install python=3.7

In [ ]:
!python -V

In [ ]:
!pip install pyamtrack

In [ ]:
!pip install lmfit

In [ ]:
# import bisect

In [ ]:
import numpy as np
from scipy import optimize
import pandas as pd

In [ ]:
np.__version__

In [ ]:
#matplotlib
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')
plt.rcParams['figure.figsize'] = [20/2.54, 15/2.54]
plt.rcParams['figure.dpi'] = 150

In [ ]:
# libamtrack
from pyamtrack import libAT

In [ ]:
# astropy needed to support units
import astropy
from astropy import units as u
from astropy import constants as const

In [ ]:
# create new units based on [MeV / u]
eV_u = u.def_unit('eV/u', u.eV / u.u)
keV_u = u.def_unit('keV/u', u.keV / u.u)
MeV_u = u.def_unit('MeV/u', u.MeV / u.u)
GeV_u = u.def_unit('GeV/u', u.GeV / u.u)
Gy = u.def_unit('Gy', u.J / u.kg)

u.add_enabled_units([eV_u, keV_u, MeV_u, GeV_u, Gy])

# shortcut for dimensionless unit
dimless = u.dimensionless_unscaled

## energy to beta

In [ ]:
@u.quantity_input
def e2beta(E_MeV_u : MeV_u, libamtrack : bool = False) -> dimless:
    """
    Convert energy in MeV/u to a beta value.

    By definition:
    E_kin = E_MeV_u * m  (assuming mass in expressed in "u" units)

    Relativistic quantity gamma can be calculated as:
    gamma = E_total / E_rest = (E_kin + E_rest) / E_rest = 1.0 + E_kin / E_rest = 1.0 + E_kin / (mc^2)

    Atomic mass unit has equivalent rest energy:
    m_u c^2 = 931.494 MeV
    Here:
    m_u = 1 * u
    m_u -> physical quantity (mass)
    1 -> unitless scalar
    u -> physical unit (atomic mass unit)
    therefore:
    c^2 = 931.494 MeV/u

    Therefore:
    gamma = 1. + E_MeV_u / c^2

    beta^2 = 1. - 1. / gamma^2 = 1. - 1. / (1. + E_MeV_u / c^2)^2
    beta = sqrt(1. - 1. / (1. + E_MeV_u / c^2)^2)

    Assuming that E_MeV_u = E_MeV_u_scalar MeV/u and knowing that c^2 = 931.494 MeV/u
    we can write in dimensionless form:
    
    beta = sqrt(1. - 1. / (1. + E_MeV_u_scalar / 931.494)^2)

    :param energy: kinetic energy [MeV/u]
    :param libamtrack: calculate using libamtrack library
    :returns: beta, v/c []
    """
    if libamtrack:
      if E_MeV_u.isscalar:
        result = libAT.AT_beta_from_E_single(E_MeV_u.to_value(MeV_u)) << dimless
      else:
        result = np.zeros(E_MeV_u.shape).tolist()
        libAT.AT_beta_from_E(E_MeV_u.to_value(MeV_u).tolist(), result)
        result <<= dimless
    else:
      result = np.sqrt(1. - (1. / (E_MeV_u / const.c**2 + 1.)**2))
    return result

## beta to energy

In [ ]:
# @u.quantity_input
# def beta2e(beta : dimless, libamtrack : bool = False) -> MeV_u:
#     """
#     Convert beta to energy in MeV/u.

#     Inverse of e2beta

#     :param beta: v/c []
#     :param libamtrack: calculate using libamtrack library
#     :returns: energy in [MeV/u]
#     """
#     if libamtrack:
#       if beta.isscalar:
#         result = libAT.AT_E_from_beta_single(beta.to_value(dimless)) << MeV_u
#       else:
#         result = np.zeros(beta.shape).tolist()
#         libAT.AT_E_from_beta(beta.to_value(dimless).tolist(), result)
#         result <<= MeV_u
#     else:
#       result = (np.sqrt(1./(1.-beta**2))-1.)* const.c**2
#     return result

## effective charge

In [ ]:
@u.quantity_input
def zeff(Z : dimless, beta : dimless, libamtrack : bool = False) -> dimless:
    """
    Effective charge

    :param Z: ion charge []
    :param beta: v/c []
    :param libamtrack: calculate using libamtrack library
    :returns: effective charge []
    """
    if not isinstance(Z, astropy.units.quantity.Quantity):
      Z <<= dimless
    if not isinstance(beta, astropy.units.quantity.Quantity):
      beta <<= dimless
    if libamtrack:
      if beta.isscalar:
        result = libAT.AT_effective_charge_from_beta_single(beta.to_value(dimless), int(Z.to_value(dimless))) << dimless
      else:
        result = np.zeros(beta.shape).tolist()
        libAT.AT_effective_charge_from_beta(beta.to_value(dimless).tolist(), [int(z) for z in Z.to_value(dimless)], result)
        result <<= dimless
    else:
      result = Z*(1. - np.exp(-125. * beta / Z**(2./3.)))
    return result

## Stopping power

In [ ]:
@u.quantity_input
def glet(E_MeV_u : MeV_u, Z : dimless, libamtrack : bool = False) -> u.keV/u.um:
    """
    Calculates the stopping power (LET) of the projectile in liquid water
    :param energy: Energy in [MeV/u]
    :param z: Charge of projectile. e.g. 6 for Carbon ions.
    :param libamtrack: calculate using libamtrack library
    :returns: LET in [keV/um]
    """
    if not isinstance(Z, astropy.units.quantity.Quantity):
      Z <<= dimless
    if Z.isscalar and not E_MeV_u.isscalar:
      Z = np.ones(E_MeV_u.shape) * Z

    if libamtrack:
      material_no = libAT.material_no.Water_Liquid.value
      stop_power_source_no = libAT.stoppingPowerSource_no.PSTAR.value
      particle_no = 1000*Z + Z
      if E_MeV_u.isscalar:
        particle_no = [int((1000*Z + Z).to_value(dimless))]
        list_E_MeV_u = [E_MeV_u.to_value(MeV_u)]
        result = [0.]
      else:
        particle_no = [int(z) for z in (1000*Z + Z).to_value(dimless)]
        list_E_MeV_u = E_MeV_u.to_value(MeV_u).tolist()
        result = np.zeros(E_MeV_u.shape).tolist()
      libAT.AT_Stopping_Power_with_no(
          libAT.stoppingPowerSource_no.PSTAR.value,
          list_E_MeV_u,
          particle_no,
          libAT.material_no.Water_Liquid.value,
          result
      )
      if E_MeV_u.isscalar:
        result = result[0]
      let_keV_um = result << u.keV / u.um
    else:
      # proton mass stopping power (in [MeV cm^2 /g]) as function of energy (in [MeV/u]) 
      # is piece-wise approximated by polynomials 
      # approximation is defined on intervals separated by following valeues: 60 keV/u, 10 MeV/u, 1 GeV/u
      p1 = np.poly1d([-0.328797e-2, -0.869548e-1, -0.911414e0, -0.471453e1, -0.115638e2, -0.374474e1])
      p2 = np.poly1d([0.649495e-3, 0.141793e-2, -0.108231e-1, 0.190340e-2, 0.886293e-3, -0.712500e0, 0.553740e1])
      p3 = np.poly1d([0.113600e-2, -0.501344e-2, -0.210655e-1, -0.670708e0, 0.549002e1])
      p4 = np.poly1d([-0.232769e-1, 0.618546e0, -0.541066e1, 0.163141e2])
      log_E_MeV_u = np.log(E_MeV_u.to_value(MeV_u)) # convert to plain floating point numbers (log of energy)
      rlet = np.piecewise(log_E_MeV_u,
                          condlist=[E_MeV_u <= 60. * keV_u, 
                                    (E_MeV_u > 60. * keV_u) & (E_MeV_u <= 10. * MeV_u),
                                    (E_MeV_u > 10. * MeV_u) & (E_MeV_u <= 1. * GeV_u),
                                    E_MeV_u > 1. * GeV_u],
                          funclist=[p1, p2, p3, p4])
      proton_mass_stop_power = np.exp(rlet) << u.MeV * u.cm**2 / u.g # apply exp function and convert back to quantity objects

      # ion mass stopping power is calculated from proton mass stopping power by rescaling by effective charge
      beta = e2beta(E_MeV_u)
      zeff_proton = zeff(Z=1, beta=beta)
      zeff_ion = zeff(Z=Z, beta=beta)
      ion_mass_stop_power = (zeff_ion / zeff_proton)**2 * proton_mass_stop_power

      # ion stopping power (LET) is calculated by scaling with water density
      water_density = 1.0 * u.g / u.cm**3
      let_keV_um = ion_mass_stop_power * water_density
    
    return let_keV_um

#RBE Wedenberg

In [ ]:
def rbe_wedenberg(dose, let, abx,q):
    """
    Wedenberg proton RBE model
    input parameters may be either numpy.array or scalars
    TODO: handle Cube() class directly
    :params dose: physical proton dose in [Gy]
    :params let: LETd in [keV/um] (protons only)
    :params abx: alpha_x / beta_x [Gy]
    :returns: RBE for the given parameters
    :ref: http://dx.doi.org/10.3109/0284186X.2012.705892
    """

    _apx = 1.000 + q * let / abx
    _sbpx = 1.0

    rbe = _rbe_apx(dose, _apx, _sbpx, abx)
    return rbe


In [ ]:
def _rbe_apx(dose, apx, sbpx, abx, dzero=0.0):
    """
    :params dose: proton dose      [Gy]
    :params apx: alpha_p / alpha_x [dimensionless] RBE_max = ap/ax when (dose -> 0 Gy)
    :params sbpx: beta_p / beta_x  [dimensionless] RBE_min = sqrt(bp/bx) when (dose -> inf Gy)
    :params abx: alpha_x / beta_x  [Gy]
    :params dzero: what to return in case of dose is zero (which would cause division by zero)
    """

    _rbe = 1.0 / (2.0 * dose)
    if hasattr(_rbe, '__iter__'):
        _rbe[_rbe == np.inf] = dzero
    else:
        if _rbe == np.inf:
            return dzero
    delta = abx * abx + 4. * abx * apx * dose + 4. * sbpx * sbpx * dose * dose
    delta *= (delta > 0)
    _rbe *= (np.sqrt(delta) - abx)
    return _rbe

#LET and Energy

In [ ]:
q = pd.read_csv("q_all_10000.csv")

In [ ]:
lets = glet( np.array(range(5,250,5)) * MeV_u, 1)

In [ ]:
lets

In [ ]:
energy =5.
ab=2.0
dose=2.0
err = []
le = []
en = []
for let in lets:
  df_=rbe_wedenberg(dose,let,ab,q.q)
  err.append(1.44*df_.std()/df_.mean() *100)
  le.append(let.value)
  en.append(energy)

  #print("energy: ",energy,"MeV ", " let: ","{:.2f}".format(let), "  error:  ","{:.2f}".format(1.44*df_.std()/df_.mean() *100),"%")
  energy=energy+5.

In [ ]:
tmp=pd.DataFrame()
tmp["Energy"] = np.round(en,2)
tmp["LET"] = le
tmp["Error"] = err

In [ ]:
energy =5.
ab=10.0
dose=2.0
err = []
le = []
en = []
for let in lets:
  df_=rbe_wedenberg(dose,let,ab,q.q)
  err.append((1.44*df_.std())/df_.mean() *100)
  le.append(let.value)
  en.append(energy)
  #print("energy: ",energy,"MeV ", " let: ","{:.2f}".format(let), "  error:  ","{:.2f}".format(1.44*df_.std()/df_.mean() *100),"%")
  energy=energy+5.

In [ ]:
tmp2=pd.DataFrame()
tmp2["Energy"] = np.round(en,2)
tmp2["LET"] = le
tmp2["Error"] = err

In [ ]:
fig=plt.figure()
ax1=fig.add_subplot(111, label="1")

ax1.plot(tmp.LET,tmp.Error, alpha=0.5,color="blue")
ax1.scatter(tmp.LET,tmp.Error,marker='.', color="black",label='ab=2.0')

ax1.plot(tmp2.LET,tmp2.Error,alpha=0.5,color="blue")
ax1.scatter(tmp2.LET,tmp2.Error,marker='x', color="black",label='ab=10.0')

#graph settings
ax1.set_title("Błąd wyznaczenia RBE", fontsize = 16)
ax1.set_xlabel("LET [keV/um]", fontsize = 12)
ax1.set_ylabel(r"$1.44\sigma/mean [%]$", fontsize = 12)

plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')

plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.top'] = True 
plt.rcParams['ytick.right'] = True

ax1.legend()
plt.show()

In [ ]:
#rozdzielic na dwa wykresy

In [ ]:
fig=plt.figure()

ax2=fig.add_subplot(111, label="1")

ax2.plot(tmp.Energy,tmp.Error, alpha=0.5,color="red")
ax2.plot(tmp2.Energy,tmp2.Error,alpha=0.5,color="red")

ax2.scatter(tmp.Energy,tmp.Error, marker='.',color="red",label="ab=2.0")
ax2.scatter(tmp2.Energy,tmp2.Error, marker='x',color="red",label='ab=10.0')


#graph settings
ax1.set_title("Błąd wyznaczenia RBE", fontsize = 16)
ax2.set_xlabel("Energy [MeV]", fontsize = 12)
ax2.set_ylabel(r"$1.44\sigma/mean [%]$",fontsize = 12)


plt.grid()
plt.minorticks_on()
plt.grid(which='minor', linestyle=':', linewidth='0.2', color='black')

plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.top'] = True 
plt.rcParams['ytick.right'] = True

ax2.legend()
plt.show()

Wielkość błędów branych pod uwagę w analizach odporności wynosi
zazwyczaj 85% CI dla niepewności konfiguracji i zakresu. 85% CI (1,44 dla normalnych rozkładów) zostało pierwotnie zaproponowane przez Goiteina (1983) i zostało przyjęte jako rozsądny parametr do wykorzystania w celach klinicznych (Albertini i wsp. 2011, Lowe i wsp. 2016, Paganetti 2012b). Zwykle skutkuje to marginesami niepewności zakresu wynoszącymi około ± 3% w klinice (Paganetti 2012b, Yang i wsp. 2012). Obliczenia błędów planu leczenia są następnie zwykle wykonywane dla skończonej liczby scenariuszy błędów sztywnego ustawienia i błędów stałej gęstości, co prowadzi do zbioru potencjalnych rozkładów dawki.